In [ ]:
%run ../00_AdvancedPythonConcepts/talktools.py

# A brief introduction to SciPy

Scipy is a collection of packages, these typically need to be imported separately:

     cluster                      : Vector Quantization / Kmeans
     fftpack                      : Discrete Fourier Transform algorithms
     integrate                    : Integration routines
     interpolate                  : Interpolation Tools
     io                           : Data input and output
     lib                          : Python wrappers to external libraries
     lib.blas                     : Wrappers to BLAS library
     lib.lapack                   : Wrappers to LAPACK library
     linalg                       : Linear algebra routines
     misc                         : Various utilities that don't have another home.
     ndimage                      : n-dimensional image package
     odr                          : Orthogonal Distance Regression
     optimize                     : Optimization Tools
     signal                       : Signal Processing Tools
     sparse                       : Sparse Matrices
     sparse.linalg                : Sparse Linear Algebra
     sparse.linalg.dsolve         : Linear Solvers
     sparse.linalg.dsolve.umfpack : :Interface to the UMFPACK library:
     sparse.linalg.eigen          : Sparse Eigenvalue Solvers
     sparse.linalg.eigen.arpack   : Eigenvalue solver using iterative methods.
     sparse.linalg.eigen.lobpcg   : Locally Optimal Block Preconditioned
                                      Conjugate Gradient Method (LOBPCG)     
     spatial                      : Spatial data structures and algorithms
     special                      : Airy Functions
     stats                        : Statistical Functions
     stats.mstats                 : Statistical functions for masked arrays
     weave                        : C/C++ integration

## Vectorizing for speed

This is silly:

In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np

def dumb_myfun(a, b):
    return np.array([a[i] if a[i] < b else 2*b for i in range(len(a))])

In [ ]:
x = np.arange(1.e6)/1.e6
x

In [ ]:
%timeit dumb_myfun(x, 0.5)

We can use `scipy.vectorize`, which is similar to `map` but operates fast on numpy arrays:

In [ ]:
import scipy as sp
myfun = lambda a,b: a if a < b else 2*b
vec_myfun = sp.vectorize(myfun)
vec_myfun

In [ ]:
%timeit vec_myfun(x, 0.5)

Let's not forget `numexpr` we saw earlier today:

In [ ]:
import numexpr as ne
b = 0.5
%timeit ne.evaluate("where(x < %f, x, %f)" % (b, b*2))

# Scipy constants

`scipy.constants` is a convenient compilation of the [2014 CODATA constants](http://physics.nist.gov/cuu/Constants/index.html).

In [ ]:
from scipy import constants as cons
print(cons.milli, cons.eV, cons.c)

In [ ]:
cons.physical_constants

In [ ]:
len(cons.physical_constants)

We can even search it like a little database:

In [ ]:
cons.find('Newton')

In [ ]:
G = cons.value('Newtonian constant of gravitation')
G_u = cons.unit('Newtonian constant of gravitation')
G_e = cons.precision('Newtonian constant of gravitation')
print('G = ', G, '±', G_e*G, G_u)

It also provides some handy unit conversions:

In [ ]:
print('32 F in K:', cons.F2K(32))
print('32 F in C:', cons.F2C(32))

# Interpolation

In [ ]:
from scipy.interpolate import interp1d, UnivariateSpline
from scipy import constants as cons

In [ ]:
# set up some fake data, listing locations versus time
rng = np.random.RandomState(42)

M = 5.98e24  # mass and radius of Earth
R = 6.38e6
accel = cons.G*M/R**2  # Earth's grav acceleration
times = np.arange(0,10,0.5)  # seconds
locations = 0.5*accel*times**2 + 50*np.random.random(len(times))

# now interpolate onto a much finer grid, using both iterp1d and UnivariateSpline
i_times = np.arange(0.5,9,0.1)
# list of interpolated values of location at i_times
i_locs = interp1d(times, locations, kind="cubic")
s = UnivariateSpline(times, locations) # a function that will return interpolated values

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

In [ ]:
# plot up several different views of this example
plt.figure(figsize=(8,8))
plt.errorbar(times, locations, yerr=50/2.35, linestyle='None', marker=".", label='data')
plt.plot(i_times, s(i_times), c="g", lw=10, alpha=0.3, label='UnivariateSpline')
plt.plot(i_times, i_locs(i_times), c="black", lw=1, alpha=0.9, label='interp1d')
plt.ylabel('locations',fontsize=15)
plt.xlabel('times (s)',fontsize=15)
plt.legend(loc='best');

## Numerical Integration

In [ ]:
from scipy.integrate import quad
val, err =quad( lambda x: np.sin(x) , 0 , np.pi, full_output=False)
print(val, err)

In [ ]:
from scipy.integrate import ode
ode?

## Special functions

In [ ]:
from scipy import special
special?

In [ ]:
from scipy.special import betainc
betainc(10, 10, 0.2)

## Polynomial Fitting

Basic (least squares) polynomial fitting can be performed using the polyfit routine. More complicated fitting tasks require scipy.

In [ ]:
x = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])
y = np.array([0.0, 0.8, 0.9, 0.1, -0.8, -1.0])

f, ax = plt.subplots()
ax.plot(x, y, 'o', label='data');
ax.legend();

Let's fit this data with a cubic polynomial

In [ ]:
z = np.polyfit(x, y, 3)
p = np.poly1d(z)
ax.plot(x, p(x), '-', lw=2, label='$p_3(x)$')
ax.set_ylim(-2, 2)
ax.legend()
f

And now let' try to fit it with a degree *30* one:

In [ ]:
p30 = np.poly1d(np.polyfit(x, y, 30))
xp = np.linspace(-2, 6, 100)
ax.plot(xp, p30(xp), '--', lw=2, label='$p_{30}(x)$')
ax.legend(fontsize=15)
f

## Optimization

A simple Least Squares Fitting example. Note that in practice, you may want to choose `fmin` which provides finer control than `leastsq`.  But for simple cases `leastsq` is sufficient: 

In [ ]:
from scipy.optimize import leastsq
from numpy import sin, cos, pi, sqrt

# Define a simple signal model
def model(par):
    p = 1.0 # constant frequency
    return par[0] + par[1]*sin(2*pi*t/p) + par[2]*cos(2*pi*t/p)

# Create some data with this model
par = [0.1, 1.5, 2]
t = np.linspace(0, pi, 300)
y = model(par)

# Add a bit of gaussian noise
dy = 0.5*np.random.rand(y.shape[0])
y += dy

# Define the residual function we minimize, simply the squared error
def resid(par):
    return (model(par)-y)**2

# And call leastsq for the fit (note that it returns a tuple whose first
# element is the parameter fit, what we actually want):
rez = leastsq(resid, [y.mean(), 1, 1] )[0]

print('Exact parameters: ', par)
print('Fitted parameters:', rez)

A quick visual verification:

In [ ]:
plt.plot(t, y, label='data')
plt.plot(t, model(rez), label='model')
plt.legend();